# ساخت با مدل‌های خانواده متا

## مقدمه

در این درس به موارد زیر می‌پردازیم:

- بررسی دو مدل اصلی خانواده متا - Llama 3.1 و Llama 3.2
- درک کاربردها و سناریوهای هر مدل
- نمونه کد برای نمایش ویژگی‌های منحصربه‌فرد هر مدل

## خانواده مدل‌های متا

در این درس، دو مدل از خانواده متا یا "گله لاما" را بررسی می‌کنیم - Llama 3.1 و Llama 3.2

این مدل‌ها در نسخه‌های مختلف ارائه شده‌اند و در بازار مدل‌های گیت‌هاب در دسترس هستند. برای اطلاعات بیشتر درباره استفاده از مدل‌های گیت‌هاب جهت [نمونه‌سازی با مدل‌های هوش مصنوعی](https://docs.github.com/en/github-models/prototyping-with-ai-models?WT.mc_id=academic-105485-koreyst) این لینک را ببینید.

انواع مدل‌ها:
- Llama 3.1 - 70B Instruct
- Llama 3.1 - 405B Instruct
- Llama 3.2 - 11B Vision Instruct
- Llama 3.2 - 90B Vision Instruct

*توجه: Llama 3 نیز در مدل‌های گیت‌هاب موجود است اما در این درس پوشش داده نمی‌شود*


## لاما ۳.۱

با داشتن ۴۰۵ میلیارد پارامتر، لاما ۳.۱ در دسته مدل‌های متن‌باز LLM قرار می‌گیرد.

این مدل نسبت به نسخه قبلی یعنی لاما ۳ ارتقا یافته و امکانات زیر را ارائه می‌دهد:

- پنجره متنی بزرگ‌تر – ۱۲۸ هزار توکن در مقابل ۸ هزار توکن
- حداکثر توکن خروجی بیشتر – ۴۰۹۶ در مقابل ۲۰۴۸
- پشتیبانی بهتر از چندزبانگی – به دلیل افزایش تعداد توکن‌های آموزشی

این ویژگی‌ها باعث می‌شوند لاما ۳.۱ بتواند در ساخت برنامه‌های GenAI موارد پیچیده‌تری را مدیریت کند، از جمله:
- فراخوانی توابع به صورت بومی – امکان فراخوانی ابزارها و توابع خارجی خارج از جریان کاری LLM
- عملکرد بهتر RAG – به دلیل پنجره متنی بزرگ‌تر
- تولید داده مصنوعی – توانایی ساخت داده‌های مؤثر برای کارهایی مانند ریزتنظیم


### فراخوانی توابع بومی

Llama 3.1 به گونه‌ای بهینه‌سازی شده که در فراخوانی توابع یا ابزارها عملکرد بهتری داشته باشد. همچنین دو ابزار داخلی دارد که مدل می‌تواند بر اساس درخواست کاربر تشخیص دهد که باید از آن‌ها استفاده کند. این ابزارها عبارتند از:

- **Brave Search** - می‌تواند برای دریافت اطلاعات به‌روز مانند وضعیت آب‌وهوا با جستجوی وب استفاده شود
- **Wolfram Alpha** - می‌تواند برای انجام محاسبات ریاضی پیچیده‌تر به کار رود تا نیازی به نوشتن توابع اختصاصی نباشد.

همچنین می‌توانید ابزارهای سفارشی خود را بسازید تا LLM بتواند آن‌ها را فراخوانی کند.

در مثال کد زیر:

- ابزارهای موجود (brave_search, wolfram_alpha) را در پیام سیستم تعریف می‌کنیم.
- یک پیام کاربر ارسال می‌شود که درباره وضعیت آب‌وهوای یک شهر خاص سؤال می‌کند.
- LLM با یک فراخوانی ابزار به Brave Search پاسخ می‌دهد که به این صورت خواهد بود: `<|python_tag|>brave_search.call(query="Stockholm weather")`

*توجه: این مثال فقط فراخوانی ابزار را انجام می‌دهد. اگر می‌خواهید نتایج را دریافت کنید، باید یک حساب رایگان در صفحه API بریو بسازید و خود تابع را تعریف کنید.*


In [ ]:
%pip install azure-core
%pip install azure-ai-inference

In [ ]:
import os
from azure.ai.inference import ChatCompletionsClient
from azure.ai.inference.models import AssistantMessage, SystemMessage, UserMessage
from azure.core.credentials import AzureKeyCredential

token = os.environ["GITHUB_TOKEN"]
endpoint = "https://models.inference.ai.azure.com"
model_name = "meta-llama-3.1-405b-instruct"

client = ChatCompletionsClient(
    endpoint=endpoint,
    credential=AzureKeyCredential(token),
)


tool_prompt=f"""
<|begin_of_text|><|start_header_id|>system<|end_header_id|>

Environment: ipython
Tools: brave_search, wolfram_alpha
Cutting Knowledge Date: December 2023
Today Date: 23 July 2024

You are a helpful assistant<|eot_id|>
"""

messages = [
    SystemMessage(content=tool_prompt),
    UserMessage(content="What is the weather in Stockholm?"),

]

response = client.complete(messages=messages, model=model_name)

print(response.choices[0].message.content)





### لاما ۳.۲

با وجود اینکه لاما ۳.۱ یک مدل زبانی بزرگ است، اما یکی از محدودیت‌های آن نبود قابلیت چندرسانه‌ای است. یعنی نمی‌تواند از ورودی‌های مختلف مثل تصویر به عنوان پرامپت استفاده کند و پاسخ بدهد. این قابلیت یکی از ویژگی‌های اصلی لاما ۳.۲ به حساب می‌آید. این ویژگی‌ها شامل موارد زیر هستند:

- چندرسانه‌ای بودن – توانایی ارزیابی همزمان پرامپت‌های متنی و تصویری را دارد
- نسخه‌های کوچک تا متوسط (۱۱ میلیارد و ۹۰ میلیارد) – این موضوع گزینه‌های استقرار منعطفی را فراهم می‌کند،
- نسخه‌های فقط متنی (۱ میلیارد و ۳ میلیارد) – این امکان را می‌دهد که مدل روی دستگاه‌های لبه یا موبایل اجرا شود و تأخیر کمی داشته باشد

پشتیبانی از چندرسانه‌ای بودن، یک گام بزرگ در دنیای مدل‌های متن‌باز به شمار می‌رود. نمونه کد زیر همزمان یک تصویر و یک پرامپت متنی را دریافت می‌کند تا تحلیلی از تصویر توسط لاما ۳.۲ نسخه ۹۰ میلیارد ارائه شود.

### پشتیبانی چندرسانه‌ای با لاما ۳.۲


In [ ]:
%pip install azure-core
%pip install azure-ai-inference

In [ ]:
import os
from azure.ai.inference import ChatCompletionsClient
from azure.ai.inference.models import (
    SystemMessage,
    UserMessage,
    TextContentItem,
    ImageContentItem,
    ImageUrl,
    ImageDetailLevel,
)
from azure.core.credentials import AzureKeyCredential

token = os.environ["GITHUB_TOKEN"]
endpoint = "https://models.inference.ai.azure.com"
model_name = "Llama-3.2-90B-Vision-Instruct"


client = ChatCompletionsClient(
    endpoint=endpoint,
    credential=AzureKeyCredential(token),
)

response = client.complete(
    messages=[
        SystemMessage(
            content="You are a helpful assistant that describes images in details."
        ),
        UserMessage(
            content=[
                TextContentItem(text="What's in this image?"),
                ImageContentItem(
                    image_url=ImageUrl.load(
                        image_file="sample.jpg",
                        image_format="jpg",
                        detail=ImageDetailLevel.LOW)
                ),
            ],
        ),
    ],
    model=model_name,
)

print(response.choices[0].message.content)

## یادگیری اینجا متوقف نمی‌شود، مسیر را ادامه دهید

بعد از اتمام این درس، مجموعه [یادگیری هوش مصنوعی مولد](https://aka.ms/genai-collection?WT.mc_id=academic-105485-koreyst) را ببینید تا دانش خود را در زمینه هوش مصنوعی مولد بیشتر کنید!



---

**سلب مسئولیت**:  
این سند با استفاده از سرویس ترجمه هوش مصنوعی [Co-op Translator](https://github.com/Azure/co-op-translator) ترجمه شده است. در حالی که ما برای دقت تلاش می‌کنیم، لطفاً توجه داشته باشید که ترجمه‌های خودکار ممکن است شامل خطا یا نادقتی باشند. نسخه اصلی سند به زبان مادری آن باید به عنوان منبع معتبر در نظر گرفته شود. برای اطلاعات حساس، ترجمه حرفه‌ای انسانی توصیه می‌شود. ما هیچ مسئولیتی در قبال سوءتفاهم یا تفسیر نادرست ناشی از استفاده از این ترجمه نداریم.
